In [1]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
from evaluate import error
import time
import re
from torchvision.transforms import Resize
from syn_DI_dataset import make_dataset, make_dataloader

## Load ALL Models

### Depth

In [17]:
from depth_benchmark.depth_ResNet18 import *
depth_model = Depth_ResNet18()
depth_model.load_state_dict(torch.load('depth_benchmark/depth_Resnet18.pt'))

<All keys matched successfully>

In [21]:
for model_component in depth_model.children():
    print(model_component)
    print('------------------')

Sequential(
  (0): Conv2d(3, 3, kernel_size=(14, 14), stride=(2, 2))
  (1): ReLU()
  (2): Conv2d(3, 3, kernel_size=(5, 56), stride=(1, 1))
  (3): ReLU()
  (4): Conv2d(3, 3, kernel_size=(5, 23), stride=(1, 1))
  (5): ReLU()
  (6): Conv2d(3, 16, kernel_size=(3, 14), stride=(1, 1))
)
------------------
Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
------------------
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
------------------
ReLU()
------------------
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
------------------
Sequential(
  (0): Block(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [11]:
depth_extractor = nn.Sequential(*list(depth_model.children())[:-2])
depth_extractor.eval()

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 3, kernel_size=(14, 14), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(5, 56), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(3, 3, kernel_size=(5, 23), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(3, 16, kernel_size=(3, 14), stride=(1, 1))
  )
  (1): Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ReLU()
  (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (5): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [18]:
class depth_feature_extractor(nn.Module):
    def __init__(self, depth_model):
        super(depth_feature_extractor, self).__init__()
        self.model = depth_model
        self.part = nn.Sequential(*list(depth_model.children())[:-2])
    def forward(self, x):
        x = self.part(x).view(x.size(0), 512, -1)
        x = x.permute(0, 2, 1)
        return x

In [20]:
depth_extractor = depth_feature_extractor(depth_model)
test = torch.rand(1, 3, 480, 640)
out = depth_extractor(test)
print(out.shape)

torch.Size([1, 49, 512])


#### check parameters （prove to be the same)

In [23]:
for param in depth_model.parameters():
    print(param)
    print(a)

Parameter containing:
tensor([[[[ 0.1939,  0.4107,  0.3786,  ...,  0.2861,  0.2946,  0.3210],
          [ 0.5559,  0.7381,  0.6227,  ...,  0.2121,  0.2000,  0.2560],
          [ 0.6243,  0.7855,  0.6152,  ...,  0.0508,  0.0452,  0.1276],
          ...,
          [ 0.3141,  0.3892,  0.2907,  ...,  0.1173,  0.1025,  0.1431],
          [ 0.1868,  0.2515,  0.1721,  ...,  0.1882,  0.2139,  0.2043],
          [ 0.1245,  0.1979,  0.1380,  ...,  0.3087,  0.3390,  0.3745]],

         [[ 0.1915,  0.4059,  0.3734,  ...,  0.3124,  0.3008,  0.3543],
          [ 0.5708,  0.7230,  0.6354,  ...,  0.1938,  0.1972,  0.2430],
          [ 0.6191,  0.7498,  0.5987,  ...,  0.0607,  0.0434,  0.1143],
          ...,
          [ 0.3030,  0.4020,  0.2892,  ...,  0.1456,  0.1044,  0.1125],
          [ 0.1954,  0.2761,  0.1812,  ...,  0.1991,  0.1991,  0.2287],
          [ 0.1310,  0.1962,  0.1254,  ...,  0.3305,  0.3420,  0.3697]],

         [[ 0.2039,  0.3938,  0.3552,  ...,  0.2810,  0.2943,  0.3201],
        

NameError: name 'a' is not defined

In [24]:
for param in depth_extractor.parameters():
    print(param)
    print(a)

Parameter containing:
tensor([[[[ 0.1939,  0.4107,  0.3786,  ...,  0.2861,  0.2946,  0.3210],
          [ 0.5559,  0.7381,  0.6227,  ...,  0.2121,  0.2000,  0.2560],
          [ 0.6243,  0.7855,  0.6152,  ...,  0.0508,  0.0452,  0.1276],
          ...,
          [ 0.3141,  0.3892,  0.2907,  ...,  0.1173,  0.1025,  0.1431],
          [ 0.1868,  0.2515,  0.1721,  ...,  0.1882,  0.2139,  0.2043],
          [ 0.1245,  0.1979,  0.1380,  ...,  0.3087,  0.3390,  0.3745]],

         [[ 0.1915,  0.4059,  0.3734,  ...,  0.3124,  0.3008,  0.3543],
          [ 0.5708,  0.7230,  0.6354,  ...,  0.1938,  0.1972,  0.2430],
          [ 0.6191,  0.7498,  0.5987,  ...,  0.0607,  0.0434,  0.1143],
          ...,
          [ 0.3030,  0.4020,  0.2892,  ...,  0.1456,  0.1044,  0.1125],
          [ 0.1954,  0.2761,  0.1812,  ...,  0.1991,  0.1991,  0.2287],
          [ 0.1310,  0.1962,  0.1254,  ...,  0.3305,  0.3420,  0.3697]],

         [[ 0.2039,  0.3938,  0.3552,  ...,  0.2810,  0.2943,  0.3201],
        

NameError: name 'a' is not defined

### mmWave

In [2]:
from mmwave_benchmark.mmwave_point_transformer_TD import *
mmwave_model = mmwave_PointTransformerReg()
mmwave_model.load_state_dict(torch.load('mmwave_benchmark/mmwave_all_random_TD.pt'))

<All keys matched successfully>

In [3]:
for model_component in mmwave_model.children():
    print(model_component)
    print('------------------')

Backbone(
  (fc1): Sequential(
    (0): Linear(in_features=5, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (transformer1): TransformerBlock(
    (fc1): Linear(in_features=32, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=32, bias=True)
    (fc_delta): Sequential(
      (0): Linear(in_features=3, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
    )
    (fc_gamma): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
    )
    (w_qs): Linear(in_features=128, out_features=128, bias=False)
    (w_ks): Linear(in_features=128, out_features=128, bias=False)
    (w_vs): Linear(in_features=128, out_features=128, bias=False)
  )
  (transition_downs): ModuleList(
    (0): Transition(
      (conv1): Sequential(
        (0): Conv1d(67, 12

In [4]:
mmwave_extractor = nn.Sequential(*list(mmwave_model.children())[:-1])
mmwave_extractor.eval()

Sequential(
  (0): Backbone(
    (fc1): Sequential(
      (0): Linear(in_features=5, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
    (transformer1): TransformerBlock(
      (fc1): Linear(in_features=32, out_features=128, bias=True)
      (fc2): Linear(in_features=128, out_features=32, bias=True)
      (fc_delta): Sequential(
        (0): Linear(in_features=3, out_features=128, bias=True)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=128, bias=True)
      )
      (fc_gamma): Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=128, bias=True)
      )
      (w_qs): Linear(in_features=128, out_features=128, bias=False)
      (w_ks): Linear(in_features=128, out_features=128, bias=False)
      (w_vs): Linear(in_features=128, out_features=128, bias=False)
    )
    (transition_downs): ModuleList(
      (0)

In [5]:
test = torch.rand(1, 57, 5)
out, _ = mmwave_extractor(test)
print(out.shape)
out = out.mean(1)
print(out.shape)

torch.Size([1, 32, 512])
torch.Size([1, 512])


### Lidar

In [1]:
from lidar_benchmark.lidar_point_transformer import *
lidar_model = lidar_PointTransformerReg()
lidar_model.load_state_dict(torch.load('lidar_benchmark/lidar_all_random.pt'))


<All keys matched successfully>

In [9]:
for model_component in lidar_model.children():
    print(model_component)
    print('------------------')

Backbone(
  (fc1): Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (transformer1): TransformerBlock(
    (fc1): Linear(in_features=32, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=32, bias=True)
    (fc_delta): Sequential(
      (0): Linear(in_features=3, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
    )
    (fc_gamma): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
    )
    (w_qs): Linear(in_features=512, out_features=512, bias=False)
    (w_ks): Linear(in_features=512, out_features=512, bias=False)
    (w_vs): Linear(in_features=512, out_features=512, bias=False)
  )
  (transition_downs): ModuleList(
    (0): TransitionDown(
      (sa): PointNetSetAbstraction(
        (mlp_

In [ ]:
print(lidar_model.transition_down1)

In [6]:
lidar_extractor = nn.Sequential(*list(lidar_model.children())[:-1])
lidar_extractor.eval()

Sequential(
  (0): Backbone(
    (fc1): Sequential(
      (0): Linear(in_features=3, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
    (transformer1): TransformerBlock(
      (fc1): Linear(in_features=32, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=32, bias=True)
      (fc_delta): Sequential(
        (0): Linear(in_features=3, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
      )
      (fc_gamma): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
      )
      (w_qs): Linear(in_features=512, out_features=512, bias=False)
      (w_ks): Linear(in_features=512, out_features=512, bias=False)
      (w_vs): Linear(in_features=512, out_features=512, bias=False)
    )
    (transition_downs): ModuleList(
      (0)

In [10]:
test = torch.rand(1, 265, 3)
out, _ = lidar_extractor(test)
out = out.mean(1)
print(out.shape)

torch.Size([1, 1024])


### WiFI-CSI

In [2]:
# from CSI_benchmark.models.mynetwork import posenet, weights_init
# csi_model = posenet()
# csi_model.load_state_dict(torch.load('CSI_benchmark/models/protocol3_random_1.pkl'))
import sys
sys.path.insert(0, './CSI_benchmark')
csi_model = torch.load('CSI_benchmark/protocol3_random_1.pkl')

In [3]:
for model_component in csi_model.children():
    print(model_component)
    print('------------------')

Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
------------------
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
------------------
ReLU(inplace=True)
------------------
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
------------------
Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2)

In [4]:
class Class_csi_extractor(nn.Module):
    def __init__(self, model):
        super(Class_csi_extractor, self).__init__()
        self.model = model
        self.part = nn.Sequential(
            model.encoder_conv1,
            model.encoder_bn1,
            model.encoder_relu,
            model.encoder_layer1,
            model.encoder_layer2,
            model.encoder_layer3,
            model.encoder_layer4, 
            torch.nn.AvgPool2d((1, 4))
        )
    def forward(self, x):
        x = torch.transpose(x, 2, 3) #16,2,114,3,32
        x = torch.flatten(x, 3, 4)# 16,2,114,96
        torch_resize = Resize([136,32])
        x = torch_resize(x)
        x = self.part(x).squeeze()
        return x

In [6]:
print(csi_model)

posenet(
  (encoder_conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (encoder_bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder_relu): ReLU(inplace=True)
  (encoder_maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (encoder_layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [5]:
csi_extractor = Class_csi_extractor(model = csi_model)
csi_extractor.eval()

Class_csi_extractor(
  (model): posenet(
    (encoder_conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (encoder_bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (encoder_relu): ReLU(inplace=True)
    (encoder_maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (encoder_layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(

In [7]:
test = torch.rand(32, 3, 114, 10).unsqueeze(1).cuda()
# print(test.shape)
# out = csi_model(test)
out = csi_extractor(test) 
print(out.shape)

torch.Size([32, 512, 17])


### RGB

In [8]:
from RGB_benchmark.rgb_ResNet18.RGB_ResNet import *

rgb_model = RGB_ResNet18()
rgb_model.load_state_dict(torch.load('./RGB_benchmark/rgb_ResNet18/RGB_Resnet18_copy.pt'))

<All keys matched successfully>

In [10]:
for model_component in rgb_model.children():
    print(model_component)
    print('------------------')

Sequential(
  (0): Conv2d(3, 3, kernel_size=(14, 14), stride=(2, 2))
  (1): ReLU()
  (2): Conv2d(3, 3, kernel_size=(5, 56), stride=(1, 1))
  (3): ReLU()
  (4): Conv2d(3, 3, kernel_size=(5, 23), stride=(1, 1))
  (5): ReLU()
  (6): Conv2d(3, 16, kernel_size=(3, 14), stride=(1, 1))
)
------------------
Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
------------------
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
------------------
ReLU()
------------------
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
------------------
Sequential(
  (0): Block(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [14]:
rgb_extractor = nn.Sequential(*list(rgb_model.children())[:-1])
rgb_extractor.eval()

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 3, kernel_size=(14, 14), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(5, 56), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(3, 3, kernel_size=(5, 23), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(3, 16, kernel_size=(3, 14), stride=(1, 1))
  )
  (1): Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ReLU()
  (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (5): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [15]:
test = torch.rand(1, 3, 480, 640)
out = rgb_extractor(test)
print(out.shape)

torch.Size([1, 512, 7, 7])


### Useless

In [3]:
from RGB_benchmark.rgb_benchmark.model import TemporalModel
checkpoint_file = 'RGB_benchmark/MMFi_RGB_benchmark_models/p3s1/epoch_60.bin'
checkpoint = torch.load(checkpoint_file)

num_joints_in = 17
num_features_in = 2
num_joints_out = 17

model_architecture = [3,3,3,3]
channels = 1024
causal = False

RGB_model = TemporalModel(num_joints_in, num_features_in, num_joints_out,
                          filter_widths=model_architecture, 
                          causal=False, dropout=0.25, channels=channels, dense=False)
RGB_model.load_state_dict(checkpoint['model_pos_train'])

<All keys matched successfully>

In [4]:
print(RGB_model)

TemporalModel(
  (drop): Dropout(p=0.25, inplace=False)
  (relu): ReLU(inplace=True)
  (expand_bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (shrink): Conv1d(1024, 51, kernel_size=(1,), stride=(1,))
  (expand_conv): Conv1d(34, 1024, kernel_size=(3,), stride=(1,), bias=False)
  (layers_conv): ModuleList(
    (0): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), dilation=(3,), bias=False)
    (1): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
    (2): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), dilation=(9,), bias=False)
    (3): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
    (4): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), dilation=(27,), bias=False)
    (5): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
  )
  (layers_bn): ModuleList(
    (0-5): 6 x BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [6]:
poses_2d_in = []
poses_3d_in = []
poses_2d_in = torch.load(os.path.join("./RGB_benchmark/rgb_benchmark/data_cache", 'p3s1', "test", "poses_2d_in.pt"))
poses_3d_in = torch.load(os.path.join("./RGB_benchmark/rgb_benchmark/data_cache", 'p3s1', "test", "poses_3d_in.pt"))

In [8]:
print(poses_3d_in.shape)

torch.Size([216, 297, 17, 3])
